In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey
import os
from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
import sim_loader as sim_load
from scipy.integrate import cumtrapz


In [ ]:
import os
import matplotlib.pyplot as plt

# Folder paths for t5 and t3
folder_path_t5 = [
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z1_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z15_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z2_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z25_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z3_Rall_t5"
]
folder_path_t3 = [
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['irradiated'] * 5
Mstar = [1.0] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_t5, planet_dict_t5 = load_simulations_and_planet_dict(folder_path_t5, H_r_model, Mstar, timestep)
Z_values_t3, planet_dict_t3 = load_simulations_and_planet_dict(folder_path_t3, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(1, 1, figsize=(8, 8))

tot_planets = 200  # Total number of planets (adjust as needed)

# Plot for t5
axs.plot(Z_values_t5, [planet_dict['SE'] / tot_planets for planet_dict in planet_dict_t5], linestyle='-', marker='o', color='brown', label='SE, $M_{\mathrm{gas, disc}}= 0.073 \: M_{\odot}$')
axs.plot(Z_values_t5, [planet_dict['sub_E'] / tot_planets for planet_dict in planet_dict_t5], linestyle='-', marker='o', color='black', label='sub-E')
axs.plot(Z_values_t5, [planet_dict['HJ'] / tot_planets for planet_dict in planet_dict_t5], linestyle='-', marker='o', color='aquamarine', label='HJ')
axs.plot(Z_values_t5, [planet_dict['WJ'] / tot_planets for planet_dict in planet_dict_t5], linestyle='-', marker='o', color='gold', label='WJ')
axs.plot(Z_values_t5, [planet_dict['CG'] / tot_planets for planet_dict in planet_dict_t5], linestyle='-', marker='o', color='purple', label='CG')

# Plot for t3
axs.plot(Z_values_t3, [planet_dict['SE'] / tot_planets for planet_dict in planet_dict_t3], linestyle='--', marker='o', color='brown', label='$M_{\mathrm{gas, disc}}= 0.065 \: M_{\odot}$')
axs.plot(Z_values_t3, [planet_dict['sub_E'] / tot_planets for planet_dict in planet_dict_t3], linestyle='--', marker='o', color='black')
axs.plot(Z_values_t3, [planet_dict['HJ'] / tot_planets for planet_dict in planet_dict_t3], linestyle='--', marker='o', color='aquamarine')
axs.plot(Z_values_t3, [planet_dict['WJ'] / tot_planets for planet_dict in planet_dict_t3], linestyle='--', marker='o', color='gold')
axs.plot(Z_values_t3, [planet_dict['CG'] / tot_planets for planet_dict in planet_dict_t3], linestyle='--', marker='o', color='purple')

# Add labels, legend, and grid
axs.legend()
axs.set_xlabel('Z', fontsize=15)
axs.set_ylabel('% of planets', fontsize=15)
axs.tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
axs.tick_params(axis="both", which="minor", direction='in', size=10)
plt.title("Planet Distribution as a Function of Dust-to-gas ratio", fontsize=18)

# Save the plot
plt.savefig("figures/pop_synth/planets_metallicity", bbox_inches='tight')
plt.show()

In [ ]:
import scipy.integrate as scipy
params = code_gas.Params(H_r_model='irradiated')
total_mass, _ = scipy.quad(M_dot_star_linear_scaling, 0.1, 5, args = (params))
print("total disc mass 5 Myr:", (total_mass*u.M_earth).to(u.M_sun))
total_mass, _ = scipy.quad(M_dot_star_linear_scaling, 0.1, 3, args = (params))
print("total disc mass 3 Myr:", (total_mass*u.M_earth).to(u.M_sun))


In [ ]:
# Folder paths for t5 and t3
folder_path_t5 = [
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z1_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z15_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z2_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z25_Rall_t5",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z3_Rall_t5"
]
folder_path_t3 = [
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar1_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['irradiated'] * 5
Mstar = [1.0] * 5  # Stellar mass in M_sun

simulations = []
sim_parameters = []
parameters = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)


simulations_t5 = []
sim_parameters_t5 = []
parameters_t5 = []

load_simulations_and_planet_dict(folder_path_t5, H_r_model, Mstar, timestep, simulations_t5, parameters_t5, sim_parameters_t5)
print(len(simulations))
fig, axs = plt.subplots(5, 2, figsize=(50, 70))

for k, (simulations, parameters, sim_parameters) in enumerate(zip(simulations_t5, parameters_t5, sim_parameters_t5)):    
    for j in range(len(simulations)):
        sim = simulations[j]
        params = parameters[j]
        sim_params = sim_parameters[j]
        for p in range(sim_params.nr_planets):
            idx = plot.idxs (axs[k,0], sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            cmap =  mpl.cm.inferno.reversed()
            norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
            colors = cmap(norm(sim_params.t0[p]))

            axs[k,0].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
            cmap = cmap)

            if iso_idx != 0:
                axs[k,0].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
                                color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

            axs[k,0].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                        color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
            
            #axs[k].scatter(sim.position[p, -1].to(u.au), Roman_Sensitivity(sim.position[p, -1].value))

    axs[k,0].set_title("Irradiated, $M_{\star}=$"+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, t3", fontsize = 18)
    plot.plot_roman_sensitivity(fig, axs[k,1], solar_system= False, ss_moons= False)
    axs[k,0].axvline(R_Einstein(8,4,params))
    print("R_Einstein", R_Einstein(8,4,params))
    plot.boxes(axs[k,0])
    # for i in range(1):
    #     axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
    #     axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
    #     axs[k].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)


simulations_t3 = []
sim_parameters_t3 = []
parameters_t3 = []

load_simulations_and_planet_dict(folder_path_t3, H_r_model, Mstar, timestep, simulations_t3, parameters_t3, sim_parameters_t3)

for k, (simulations, parameters, sim_parameters) in enumerate(zip(simulations_t3, parameters_t3, sim_parameters_t3)):    
    for j in range(len(simulations)):
        sim = simulations[j]
        params = parameters[j]
        sim_params = sim_parameters[j]
        for p in range(sim_params.nr_planets):
            idx = plot.idxs (axs[k,1], sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                            sim.dR_dt[p], sim.dM_dt[p], params, True)
            iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
            stop_mig_idx = idx['stop_mig_idx'].values[0]
            cmap =  mpl.cm.inferno.reversed()
            norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
            colors = cmap(norm(sim_params.t0[p]))

            axs[k,1].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
            cmap = cmap)

            if iso_idx != 0:
                axs[k,1].scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
                                color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')

            axs[k,1].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                        color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
            
            #axs[k].scatter(sim.position[p, -1].to(u.au), Roman_Sensitivity(sim.position[p, -1].value))

    axs[k,1].set_title("Irradiated, $M_{\star}=$"+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, t5", fontsize = 18)
    plot.plot_roman_sensitivity(fig, axs[k,1], solar_system= False, ss_moons= False)
    axs[k,1].axvline(R_Einstein(8,4,params))
    print("R_Einstein", R_Einstein(8,4,params))
    plot.boxes(axs[k,1])
    # for i in range(1):
    #     axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
    #     axs[k].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
    #     axs[k].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)


#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2
for j in range(5):
    for i in range(2):
        params = parameters[j]
        sim_params = sim_parameters[j]

        axs[j,i].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
        axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
        axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

        axs[j,i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
        axs[j,i].set_xlim(5e-3, 1e2)
        axs[j,i].tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
        axs[j,i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
        axs[j,i].set_ylim(1e-8, 1e3)
        axs[j,i].set_xscale('log')
        axs[j,i].set_yscale('log')
        # axs[j].yaxis.set_major_locator(mticker.LogLocator(numticks=999))
        # axs[j].yaxis.set_minor_locator(mticker.LogLocator(numticks=999, subs="auto"))
        plot.all_x_ticks(axs[j,i], num_ticks=100)

    axs[j,0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_single_out_t3", bbox_inches='tight')

In [ ]:
def normalize_rgb(rgb):
    return tuple([v / 255 for v in rgb])


In [ ]:
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
# Folder paths for t5 and t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun
mdot = ['linear', 'Liu et al. 2019', 'quadratic']
yel = (254, 205, 101)
br = (200, 113, 71)
bl = (186, 225, 249)
rd = (226, 94, 60)
rd2 = (192, 34, 26)
panna = (254, 240, 211)

Z_values = [0.01, 0.015, 0.02, 0.025, 0.03]  # Example metallicity values corresponding to the colors
colors = ['gold','tomato', 'teal','mediumblue', 'darkorchid']
a_p0 = np.geomspace(1e-3, 1e2, 100)
fig, axs = plt.subplots(1, 3, figsize=(22, 6))

for f, folder_path in enumerate([folder_path_linear, folder_path_liu, folder_path_quadratic]):
    simulations = []
    sim_parameters = []
    parameters = []

    load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)

    for k, (simulations, parameters, sim_parameters) in enumerate(zip(simulations, parameters, sim_parameters)):    
        for j in range(len(simulations)):
            sim = simulations[j]
            params = parameters[j]
            sim_params = sim_parameters[j]
            for p in range(sim_params.nr_planets):
                idx = plot.idxs (axs[f], sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                                sim.dR_dt[p], sim.dM_dt[p], params, True)
                iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
                stop_mig_idx = idx['stop_mig_idx'].values[0]
                color = colors[k] #color coded according to the Z values of the folder
                axs[f].scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = 'lightgrey')

                axs[f].scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), color = color)
                
                #axs[f].scatter(sim.position[p, -1].to(u.au), Roman_Sensitivity(sim.position[p, -1].value))

        axs[f].set_title("Surfheat, $M_{\star}=$"+str(params.star_mass*u.M_earth.to(u.M_sun))+" $M_{\odot}$, $\dot{M}$ "+str(mdot[f]), fontsize = 18)
    plot.plot_roman_sensitivity(fig, axs[f], solar_system= False, ss_moons= False, kepler = False)
    axs[f].axvline(R_Einstein(8,4,params), color = "black")
    print("R_Einstein", R_Einstein(8,4,params))
    plot.boxes(axs[f])
    axs[f].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_in, params), color = "slateblue", linestyle =':', zorder = 0)
    axs[f].loglog(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), color = "slateblue", linestyle =':', zorder = 0)
    axs[f].fill_between(a_p0, M_peb_iso(a_p0, sim_params.t_fin, params), M_peb_iso(a_p0, sim_params.t_in, params),  color='slateblue', alpha=0.1)    #plot the magnetic cavity and shade the region inside it (from initial to final position)

import matplotlib as mpl

# Define your custom color list and corresponding Z values
colors = ['gold', 'tomato', 'teal', 'mediumblue', 'darkorchid']
Z_values = [0.01, 0.015, 0.02, 0.025, 0.03]  # Example metallicity values corresponding to the colors

# Create a colormap from your color list
cmap = mpl.colors.ListedColormap(colors)

# Define boundaries for the color bins
boundaries = [Z_values[0] - (Z_values[1] - Z_values[0]) / 2]  # Start boundary
boundaries += [(Z_values[i] + Z_values[i + 1]) / 2 for i in range(len(Z_values) - 1)]  # Midpoints
boundaries.append(Z_values[-1] + (Z_values[-1] - Z_values[-2]) / 2)  # End boundary

# Normalize the Z values to the range [0, 1] for the colorbar
norm = mpl.colors.BoundaryNorm(boundaries=boundaries, ncolors=len(colors))

# Add the colorbar to the figure
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # Required for ScalarMappable

# Add the colorbar to the figure, positioned to the right of all subplots
cbar = fig.colorbar(sm, ax=axs, orientation='vertical', pad=0.02, aspect=20, shrink=0.8)

# Set custom ticks and labels
cbar.set_ticks(Z_values)  # Set ticks at the Z_values
cbar.set_ticklabels([f"{z:.3f}" for z in Z_values])  # Format the tick labels
cbar.set_label('Disc Metallicity (Z)', fontsize=25)
cbar.ax.tick_params(labelsize=15)

alpha_transp=0.2
for i in range(3):
    params = parameters[j]
    sim_params = sim_parameters[j]

    axs[i].axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[i].axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
    axs[i].axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

    axs[i].set_xlabel('r [AU]', fontsize = 25, labelpad=20)
    axs[i].set_xlim(3e-3, 1e2)
    axs[i].tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
    axs[i].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
    axs[i].set_ylim(1e-8, 1e3)
    axs[i].set_xscale('log')
    axs[i].set_yscale('log')
    plot.all_x_ticks(axs[i], num_ticks=100)
    plot.all_y_ticks(axs[i], num_ticks=100)

axs[0].set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_single_diff_metallicity_color_surfheat", bbox_inches='tight')

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for t5 and t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['irradiated'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='aquamarine', label='irradiation' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#FF33FF', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#E28D8D', label=str(label[i]))


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)


folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]

timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

dictionaries = [planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-','--', '-.']  # Line styles for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#00CC88', label='viscous heating' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='#B89C00', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='brown')

#lower mass M dwarfs!
axs[0].axhline(0.52, color= 'gold', linestyle='--', label='Bonfis et al. 2013')
axs[0].fill_betweenx([1.02, 0.36], 0.01, 0.03, color='gold', alpha=0.1)
axs[0].axhline(0.55, color= 'orange', linestyle='--', label='Sabotta et al. 2015')
axs[0].fill_betweenx([0.96, 0.29], 0.01, 0.03, color='orange', alpha=0.1)

#0.3<M<0.7 M_sun M-dwarfs
axs[0].axhline(0.85, color= 'lightgreen', linestyle='--', label='Pinamonti et al. 2022')
axs[0].fill_betweenx([1.31, 0.64], 0.01, 0.03, color='lightgreen', alpha=0.1)
axs[1].legend()
# Add a title for the entire figure
#fig.suptitle("Planets as a Function of Z and $\dot{M}_{\star}$ for $M_{\star} = 0.5 \: M_{\odot}$", fontsize=18)
# Save the plot
plt.savefig("figures/pop_synth/planets_metallicity_mdot_scaling_nosubE", bbox_inches='tight')
plt.show()

In [ ]:
import os
import matplotlib.pyplot as plt
# Folder paths for t5 and t3
folder_path_linear = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/Mstar05_Z3_Rall_t3"
]
folder_path_quadratic = [
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/quadratic/Mstar05_Z3_Rall_t3"
]
folder_path_liu = [
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z1_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z15_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z2_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z25_Rall_t3",
    "sims/gas_acc/stellar_masses/single_planets/liu/surfheat/Mstar05_Z3_Rall_t3"
]


timestep = 10000
H_r_model = ['Lambrechts_mixed'] * 5
Mstar = [0.5] * 5  # Stellar mass in M_sun

# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep):
    simulations = []
    sim_parameters = []
    parameters = []
    planet_dicts = []
    Z_values = []

    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('N_steps'+str(timestep)+'_Mstar_'+str(mstar)+'.json')]

        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        parameters.append([sim_load.load_params(filename) for filename in params_filenames])

        # Extract Z from the params file
        if parameters:
            Z_values.append(parameters[-1][0].Z)  # Assuming Z is consistent across all params files in the folder

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))


    # Sort Z values and corresponding planet_dicts
    sorted_indices = sorted(range(len(Z_values)), key=lambda i: Z_values[i])
    print("sorted indices:", sorted_indices)
    Z_values = [Z_values[i] for i in sorted_indices]
    planet_dicts = [planet_dicts[i] for i in sorted_indices]

    return Z_values, planet_dicts

# Load data for t5 and t3
Z_values_linear, planet_dict_linear = load_simulations_and_planet_dict(folder_path_linear, H_r_model, Mstar, timestep)
#Z_values_quadratic, planet_dict_quadratic = load_simulations_and_planet_dict(folder_path_quadratic, H_r_model, Mstar, timestep)
Z_values_liu, planet_dict_liu = load_simulations_and_planet_dict(folder_path_liu, H_r_model, Mstar, timestep)

# Plot planet_dict as a function of Z
fig, axs = plt.subplots(4, 1, figsize=(5, 16), sharex=True)
fig.subplots_adjust(hspace=0)
tot_planets = 200  # Total number of planets (adjust as needed)
dictionaries = [planet_dict_linear,planet_dict_liu] #[planet_dict_linear, planet_dict_quadratic, planet_dict_liu]
Z_values =  [Z_values_linear, Z_values_liu] #[Z_values_linear, Z_values_quadratic, Z_values_liu]
# Plot for t5
ls = ['-', '--', '-.']  # Line styles for different models
label = ['$\dot{M}_{\star}$ linear', '$\dot{M}_{\star}$ quadratic', '$\dot{M}_{\star}$ Liu et al.2019']  # Labels for different models
for i, (dictionaries, Z_val) in enumerate(zip(dictionaries, Z_values)):
    axs[1].plot(Z_val, [planet_dict['HJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='aquamarine', label='HJ' if i == 0 else None)
    axs[2].plot(Z_val, [planet_dict['WJ'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='gold', label='WJ' if i == 0 else None)
    axs[3].plot(Z_val, [planet_dict['CG'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='purple', label='CG' if i == 0 else None)
    axs[0].plot(Z_val, [planet_dict['SE'] / tot_planets for planet_dict in dictionaries], linestyle=ls[i], marker='o', color='brown', label=str(label[i]))


# Add labels, legend, and grid
axs[0].legend()
axs[3].set_xlabel('Z', fontsize=25)
for i in range(4):
    axs[i].tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
    axs[i].tick_params(axis="both", which="minor", direction='in', size=10)
    axs[i].grid(True, linestyle='--', alpha=0.5)

axs[0].set_ylabel('super-Earths', fontsize=20)
axs[1].set_ylabel('Hot Jupiters', fontsize=20)
axs[2].set_ylabel('Warm Giants', fontsize=20)
axs[3].set_ylabel('Cold Giants', fontsize=20)

def Z_to_Fe_H (Z):
    """Fe/H to Z relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return np.log10(Z/f_dtf_solar)

def Fe_H_to_Z (Fe_H):
    """Z to Fe/H relation according to Burn 2021"""
    f_dtf_solar = 0.0149 #Lodders 2003
    return f_dtf_solar*10**Fe_H

ticks = [Z_to_Fe_H(z) for z in Z_values_linear]  # Convert Z values to Fe/H
# Add secondary x-axis to the top subplot
secax = axs[0].secondary_xaxis('top')
secax.set_xticks(Z_values_linear)  # Set ticks at the Z values
secax.set_xticklabels([f'{val:.2f}' for val in ticks])  # Format Fe/H labels
secax.set_xlabel('[Fe/H]', fontsize=15, labelpad=10)
secax.tick_params(axis="both", which="major", direction='in',size=15, labelsize=18)
# Add shared x-axis label
axs[-1].set_xlabel('Metallicity (Z)', fontsize=15, labelpad=10)


# Add a title for the entire figure
#fig.suptitle("Planets as a Function of Z and $\dot{M}_{\star}$ for $M_{\star} = 0.5 \: M_{\odot}$", fontsize=18)
# Save the plot
plt.savefig("figures/pop_synth/planets_metallicity_mdot_scaling_nosubE_surf", bbox_inches='tight')
plt.show()

In [ ]:
mu = 0.07  # Mean [Fe/H]
sigma = 0.21  # Standard deviation
num_samples = 10000  # Number of Monte Carlo samples

# Generate random Z values from a Gaussian distribution
Fe_H_samples = np.random.normal(mu, sigma, num_samples)

fig,axs = plt.subplots(1, 1, figsize=(8, 6))
plt.hist(Fe_H_samples, bins=50, density=True, alpha=0.6, color='blue', edgecolor='black')
axs.axvline(mu, color = 'red', linestyle='--', label='Mean [Fe/H] = -0.07')
axs.axvline(mu+sigma, color = 'red', linestyle='--', label='Mean [Fe/H] = -0.07')
axs.axvline(mu-sigma, color = 'red', linestyle='--', label='Mean [Fe/H] = -0.07')
axs.set_title('Distribution of [Fe/H] Samples', fontsize=18)
plt.savefig('figures/pop_synth/FeH_distribution', bbox_inches='tight')
Z_samples = Fe_H_to_Z(Fe_H_samples)  # Convert Fe/H samples to Z
fig,axs = plt.subplots(1, 1, figsize=(8, 6))
plt.hist(Z_samples, bins=50, density=True, alpha=0.6, color='blue', edgecolor='black')
axs.axvline(0.01, color = 'red', linestyle='--', label='Mean Z = 0.0149')
axs.set_title('Distribution of Z Samples', fontsize=18)
plt.savefig('figures/pop_synth/FZ_distribution', bbox_inches='tight')

In [ ]:
M_star = np.geomspace(2e-2, 1e2, 100)  # Define the range of stellar masses
imf = []  # Use a Python list to store the IMF values
imf2 = []  # Use a Python list to store the IMF values for the second mass range
# Loop through M_star and calculate the IMF for each value
for m in M_star:
    imf.append(Kroupa_IMF_pdf(m))
    imf2.append(Chabrier_IMF_pdf(m))  # Example for a different IMF range
# Convert the list to a NumPy array
imf = np.array(imf)
imf2 = np.array(imf2)
# Plot the IMF
fig, axs = plt.subplots(1, 1, figsize=(8, 6))
axs.scatter(M_star, imf, color='blue', label='Kroupa IMF')
axs.scatter(M_star, imf2, color='orange', linestyle='--', label='Chabrier IMF')
axs.set_xlabel('Stellar Mass ($M_\odot$)', fontsize=12)
axs.set_ylabel('IMF', fontsize=12)
axs.set_title('Stellar IMF', fontsize=14)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
plt.grid(True)
plt.show()

In [ ]:
M_star = np.geomspace(0.01, 150, 100)  # Define the range of stellar masses
imf = []  # Use a Python list to store the IMF values
imf2 = []  # Use a Python list to store the IMF values for the second mass range
imf3 = []
imf4 = []  # Use a Python list to store the IMF values for the fourth mass range
imf5 = []  # Use a Python list to store the IMF values for the fifth mass range
# Loop through M_star and calculate the IMF for each value
for m in M_star:
    imf.append(Kroupa_IMF_cdf(m))
    imf2.append(Kroupa_IMF_pdf(m)) 
    imf3.append(Chabrier_2003_IMF_pdf(m)) 
    imf4.append(Chabrier_IMF_pdf(m))  
    imf5.append(Chabrier_2005_IMF_pdf(m))  # Example for a different IMF range


# Convert the list to a NumPy array
imf = np.array(imf)
imf2 = np.array(imf2)
# Plot the IMF
fig, axs = plt.subplots(1, 1, figsize=(8, 6))
# axs.scatter(M_star, imf, color='blue', label='Kroupa CDF')
axs.scatter(M_star, imf2, color='gold', label='Kroupa IMF')
# axs.scatter(M_star, imf3, color='orange', label='Chabrier IMF')
# axs.scatter(M_star, imf4, color='green', label='Chabrier 2003 IMF')
axs.scatter(M_star, imf5, color='red', label='Chabrier 2005 IMF')

axs.set_xlabel('Stellar Mass ($M_\odot$)', fontsize=12)
axs.set_ylabel('IMF', fontsize=12)
axs.set_title('Stellar IMF', fontsize=14)
axs.set_xscale('log')
axs.set_yscale('log')
axs.legend()
plt.grid(True)
plt.savefig('figures/pop_synth/IMF', bbox_inches='tight')

In [ ]:
mu = -0.07  # Mean [Fe/H]
sigma = 0.21  # Standard deviation
num_samples = 200  # Number of Monte Carlo samples
# Generate random Z values from a Gaussian distribution
Fe_H_samples = np.random.normal(mu, sigma, num_samples)
Z_samples = Fe_H_to_Z(Fe_H_samples)  # Convert Fe/H samples to Z
print("Z samples:", Z_samples)

In [ ]:
from scipy.integrate import cumtrapz

nm = 1000
which = 'Chabrier2005' #'Kroupa'

Mstars = np.logspace(-2, 2, nm)
IMF_pdf = np.zeros(nm)
MC_random = np.random.uniform(0, 1, nm)

for i in range(0, nm):
    if which == 'Chabrier2005':
        IMF_pdf[i] = Chabrier_2005_IMF_pdf(Mstars[i])
    if which == 'Kroupa':
        IMF_pdf[i] = Kroupa_IMF_pdf(Mstars[i])    

# Assume x is your array (can be linear or log-spaced), pdf is the unnormalized PDF
dx = np.diff(Mstars)
dx = np.append(dx, dx[-1])  # Make dx same length as x
# Compute normalization constant (area under curve)
area = np.sum(IMF_pdf * dx)
# Normalize
IMF_pdf_norm = IMF_pdf / area

# Now compute mean
average_pdf = np.sum(Mstars * IMF_pdf_norm * dx)



IMF_cdf = cumtrapz(IMF_pdf_norm, Mstars, initial=0)
IMF_cdf /= IMF_cdf[-1]

sampled_cdf = np.interp(MC_random, IMF_cdf, Mstars)

average_cdf = np.mean(sampled_cdf)

sampled_cdf

In [ ]:
# Plot histogram of samples (recovered PDF)
plt.figure(figsize=(8,5))

bins = np.logspace(np.log10(Mstars.min()), np.log10(Mstars.max()), 20)

plt.hist(sampled_cdf, bins=bins, density=True, histtype='step', label="Recovered PDF (histogram)", color='#DECDF5', lw=3)

# plt.axvline(average_pdf,color='#BD4C58',ls='--')
# plt.axvline(average_cdf,color='#DECDF5',ls='--')

# Also plot original PDF for comparison (optional)
plt.plot(Mstars, IMF_pdf_norm, label="Original PDF", color='#BD4C58', lw=3, alpha=0.7)

plt.xlabel("Stellar Mass")
plt.ylabel("Probability Density")
plt.title("Monte Carlo Sampling vs Original PDF")
plt.legend()
plt.xscale("log")  # because your x is logspaced
plt.savefig("figures/pop_synth/IMF_sampling", bbox_inches='tight')
 

In [ ]:
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, simulations, parameters, sim_parameters):
    for folder_path, H_r_model  in zip(folder_paths, H_r_model):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        print("all files in folder:", len(all_files))
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model) and f.endswith('.json')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) and f.endswith('.json')]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model) and f.endswith('.json')]
        print("sim_filenames:", len(sim_filenames))
        # Sort the filenames based on the initial time
        # sim_filenames.sort(key=sim_load.extract_initial_time)
        # sim_params_filenames.sort(key=sim_load.extract_initial_time)
        # params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)

folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/random_Mstar_random_Z"
]


H_r_model = ['irradiated']

fig, axs = plt.subplots(1, 1, figsize=(6, 6))

simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, simulations, parameters, sim_parameters)

Mstars = [(parameters[0][i].star_mass*u.M_earth).to(u.M_sun).value for i in range(len(parameters[0]))]  # Extract star masses from parameters
bins = np.logspace(np.log10(min(Mstars)), np.log10(max(Mstars)), 20)

print("Number of simulations loaded:", len(parameters[0]))
plt.hist([(parameters[0][i].star_mass*u.M_earth).to(u.M_sun) for i in range(len(parameters[0]))], bins = bins, edgecolor='black', color = 'red', density = True)
plt.xscale("log")  # because x is logspaced
plt.title("Distribution of Stellar Masses")
plt.xlabel("Stellar Mass ($M_\odot$)")
plt.ylabel("Probability Density")
plt.savefig("figures/pop_synth/pop_synth_star_masses", bbox_inches='tight')

## MC stellar mass and Z

In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/random_Mstar_random_Z",
]


timestep = 5000
H_r_model = ['irradiated']*2000
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))

for j in range(len(simulations[0])):
    print(j)
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
norm = mpl.colors.LogNorm(vmin=sim_params.t_in, vmax=sim_params.t_fin)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('Accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, random $M_{\star}$, random Z", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random", bbox_inches='tight')

In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/multiplanet/linear/irradiated/test",
]


timestep = 5000
H_r_model = ['irradiated']*250
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))
nr_systems = len(simulations[0])
nr_systems = 20
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
cmap = mpl.cm.nipy_spectral
norm = mpl.colors.Normalize(vmin=0, vmax=nr_systems)
colors = cmap(np.linspace(0, 1, nr_systems)) # Generate 250 colors
print("colors:", colors)

for j in range(nr_systems):
    print(j)
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    col = colors[j]
    for p in range(sim_params.nr_planets):
        print(p)
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none', edgecolors = col)
        #, norm=norm, cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color = col)
                    #color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
        axs.loglog(sim.position[p,:].to(u.au), sim.mass[p,:].to(u.M_earth), color = col, alpha = 0.5)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
# cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
# cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
# cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('System number', fontsize=20, labelpad=15)

cbar.set_label('System number', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, $M_{\star}=1 \: M_{\odot}$ , random Z, 4planets", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random_multiple_20", bbox_inches='tight')

In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar1_random_Z",
]


timestep = 5000
H_r_model = ['irradiated']*250
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))

for j in range(len(simulations[0])):
    print(j)
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        print(p)
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
norm = mpl.colors.LogNorm(vmin=sim_params.t_in, vmax=sim_params.t_fin)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('Accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, $M_{\star}=1 \: M_{\odot}$ , random Z, 1 planet", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random_single", bbox_inches='tight')

In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/irradiated/Mstar1_random_Z_random_tau_disc",
]


timestep = 5000
H_r_model = ['irradiated']*250
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))

for j in range(len(simulations[0])):
    print(j)
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        print(p)
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
norm = mpl.colors.LogNorm(vmin=sim_params.t_in, vmax=sim_params.t_fin)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('Accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, $M_{\star}=1 \: M_{\odot}$ , random Z, random $M_{\mathrm{disc}}$, 1 planet", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random_random_tau_single", bbox_inches='tight')

In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/liu/irradiated/Mstar1_random_Z_random_tau_disc",
]


timestep = 5000
H_r_model = ['irradiated']*250
Mstar = [0.5]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation_'+H_r_model)]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params_'+H_r_model) ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params_'+H_r_model)]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8,8))

for j in range(len(simulations[0])):
    print(j)
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        print(p)
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]
        cmap =  mpl.cm.inferno.reversed()
        norm = mpl.colors.LogNorm(vmin = sim_params.t_in, vmax = sim_params.t_fin)
        colors = cmap(norm(sim_params.t0[p]))

        sc = axs.scatter(sim.position[p,0].to(u.au), sim.mass[p,0].to(u.M_earth), facecolors='none',edgecolors = colors, norm=norm, 
        cmap = cmap)

        # if iso_idx != 0:
        #     axs.scatter(sim.position[p,iso_idx].to(u.au), sim.mass[p,iso_idx].to(u.M_earth),
        #                     color =  cmap(norm(sim.time[iso_idx].to(u.Myr).value)), facecolors='none', marker = 'v')
        axs.scatter(sim.position[p,stop_mig_idx].to(u.au), sim.mass[p,stop_mig_idx].to(u.M_earth), 
                    color =cmap(norm(sim.time[stop_mig_idx].to(u.Myr).value)))
# Define the colormap and normalization
cmap = mpl.cm.inferno.reversed()
norm = mpl.colors.LogNorm(vmin=sim_params.t_in, vmax=sim_params.t_fin)

# Create a standalone colorbar
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])  # Position of the colorbar
cbar = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap, norm=norm)

# Customize the colorbar
cbar_ax.yaxis.set_major_locator(mpl.ticker.LogLocator(base=10.0, subs=[1.0, 5.0]))
cbar_ax.yaxis.set_major_formatter(mpl.ticker.LogFormatter())
cbar.ax.yaxis.set_major_formatter(plt.FuncFormatter(plot.custom_log_formatter))
cbar.set_label('Accretion timescale [Myr]', fontsize=20, labelpad=15)
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)

plot.boxes(axs)
plot.plot_roman_sensitivity(fig, axs, solar_system= False, ss_moons= False, kepler=False)
#plot the initial mass line
from matplotlib import pyplot as plt, ticker as mticker
alpha_transp=0.2

# params = parameters[0]
# sim_params = sim_parameters[0]

# axs.axvline(r_magnetic_cavity(sim_params.t_in, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvline(r_magnetic_cavity(sim_params.t_fin, params), linestyle = '-.', color = 'grey', alpha = 0.1)
# axs.axvspan(r_magnetic_cavity(sim_params.t_in, params), r_magnetic_cavity(sim_params.t_fin, params),facecolor='none', hatch='/', edgecolor='gray', alpha =alpha_transp)

axs.set_xlabel('r [AU]', fontsize = 25, labelpad=20)
axs.set_xlim(5e-3, 1e2)
axs.tick_params(axis = "both", which = "major", direction = 'in', size = 15, labelsize = 18)
axs.tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
axs.set_ylim(1e-8, 1e3)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_title("Irradiated, liu,  $M_{\star}=1 \: M_{\odot}$ , random Z, random $M_{\mathrm{disc}}$, 1 planet", fontsize = 18)

plot.all_x_ticks(axs, num_ticks=100)

axs.set_ylabel('M [$M_{\oplus}$]', fontsize = 25, labelpad=20)
plt.savefig("figures/pop_synth/pop_synt_starmass_Z_random_random_tau_single_liu", bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(8, 6))
smap = np.loadtxt('sims/gas_acc/all.magrid.NRO.layout_7f_3_covfac.52.filled') 
x = 10**smap[:33,0]
y = 10**smap[::33,1]
print(smap.shape,x.shape[0]*y.shape[0])
X,Y=np.meshgrid(x,y)

z = 10**smap[:,2].reshape(X.shape) 
print("x",x)
print("y",y[::-1])
print("z", z.shape)

final_positions = []
final_masses = []
for j in range(len(simulations[0])):
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        final_positions.append(sim.position[p,stop_mig_idx].to(u.au).value)
        final_masses.append(sim.mass[p,stop_mig_idx].to(u.M_earth).value) 


import seaborn as sns
from matplotlib.colors import LogNorm

# Create the 2D histogram
# Extend the bin edges for x and y
x_bins = np.append(x, x[-1] * 1.1)  # Add an extra bin edge for x
y_bins = np.append(y[::-1], y[::-1][-1] * 1.1)  # Add an extra bin edge for y
counts, _, _ = np.histogram2d(final_positions, final_masses, bins=[x_bins, y_bins])
counts = np.fliplr(counts)# flips the count left
print(counts.shape)
pc = axs.pcolormesh(X, Y, z, cmap='viridis', shading='auto', norm = LogNorm())  # Transpose counts to match X, Y
cbar = plt.colorbar(pc, ax=axs, label='Number of Planets detection if 1 planet per star')# # Ensure z matches the histogram grid

axs.set_xlabel('r [AU]', fontsize=25, labelpad=20)
axs.set_ylabel('M [$M_{\oplus}$]', fontsize=25, labelpad=20)
axs.set_xscale('log')
axs.set_yscale('log')
axs.tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
axs.tick_params(axis="both", which="minor", direction='in', size=10)
axs.set_title("Roman Sensitivity", fontsize=18)
axs.scatter(final_positions, final_masses, alpha=0.5, edgecolor='white')
plt.savefig("figures/pop_synth/planet_count", bbox_inches='tight')

fig, axs = plt.subplots(1, 1, figsize=(8, 6))

x_bins = np.append(x, x[-1] * 1.1)  # Add an extra bin edge for x
y_bins = np.append(y[::-1], y[::-1][-1] * 1.1)  # Add an extra bin edge for y
counts, _, _ = np.histogram2d(final_positions, final_masses, bins=[x_bins, y_bins])
counts = np.fliplr(counts)# flips the count left
print(counts.shape)
pc = axs.pcolormesh(X, Y, counts.T*z, cmap='viridis', shading='auto', norm = LogNorm())  # Transpose counts to match X, Y
cbar = plt.colorbar(pc, ax=axs, label='Number of Planets detection if 1 planet per star')# # Ensure z matches the histogram grid

axs.set_xlabel('r [AU]', fontsize=25, labelpad=20)
axs.set_ylabel('M [$M_{\oplus}$]', fontsize=25, labelpad=20)
axs.set_xscale('log')
axs.set_yscale('log')
axs.tick_params(axis="both", which="major", direction='in', size=15, labelsize=18)
axs.tick_params(axis="both", which="minor", direction='in', size=10)
axs.set_title("Roman Sensitivity", fontsize=18)


# #Contours in log sensitivity
# cf = axs.contourf(X,Y,z.T,cmap='Blues',levels=[-1,-0.5,0,0.5,1,1.5,2,2.5,3,3.5,4,4.5],vmin=-1,vmax=8)
# cbar = plt.colorbar(cf,ax=axs,label='$Roman$ Sensitivity $-$ the number of planet detections\n expected if there is 1 planet per star at $(a,M_{\\rm p})$',ticks=[-1,0,1,2,3,4])
# cbar.ax.set_yticklabels(['0.1','1','10','100','1000','10000'])

plt.savefig("figures/pop_synth/weighted_planet_count", bbox_inches='tight')




In [ ]:
# Folder paths for t5 and t3
folder_path = [
    "sims/gas_acc/stellar_masses/single_planets/linear/surfheat/lowres/Fe_H_07/5Myrs_randomMstar_randomZ",
]


timestep = 500
H_r_model = ['Lambrechts_mixed']*1000
Mstar = [1.]  # Stellar mass in M_sun
simulations = []
sim_parameters = []
parameters = []
planet_dicts = []
# Function to load simulations and compute planet_dict
def load_simulations_and_planet_dict(folder_paths, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters):
    for folder_path, H_r_model, mstar in zip(folder_paths, H_r_model, Mstar):
        # List all files in the given folder
        all_files = os.listdir(folder_path)
        # Create the list of names of the sim, sim_params, and params files
        sim_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('simulation')]
        sim_params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('sim_params') ]
        params_filenames = [os.path.join(folder_path, f) for f in all_files if f.startswith('params')]
        print("Number of files in folder:", len(all_files))
        print("Number of simulation files:", len(sim_filenames))
        # Sort the filenames based on the initial time
        sim_filenames.sort(key=sim_load.extract_initial_time)
        sim_params_filenames.sort(key=sim_load.extract_initial_time)
        params_filenames.sort(key=sim_load.extract_initial_time)

        # Load the simulations, sim_params, and params
        simulations.append([sim_load.JSONtoSimRes(filename) for filename in sim_filenames])
        sim_parameters.append([sim_load.load_sim_params(filename) for filename in sim_params_filenames])
        params = [sim_load.load_params(filename) for filename in params_filenames]
        parameters.append(params)
        print("Number of simulations loaded:", len(simulations[-1]))

        # Compute planet_dict for the folder
        planet_dicts.append(planet_counter(simulations[-1], parameters[-1], sim_parameters[-1], outer=False))
        print('planet dict:', planet_dicts[-1])



simulations = []
sim_parameters = []
parameters = []
load_simulations_and_planet_dict(folder_path, H_r_model, Mstar, timestep, simulations, parameters, sim_parameters)
print(len(simulations[0]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# final positions and masses of the simulation loaded above
final_positions = []
final_masses = []
for j in range(len(simulations[0])):
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        final_positions.append(sim.position[p,stop_mig_idx].to(u.au).value)
        final_masses.append(sim.mass[p,stop_mig_idx].to(u.M_earth).value) 



# Define the grid bins for radius and mass
radius_bins = np.logspace(-2, 2, 10)  # Log-spaced bins for radius (0.01 to 100  AU)
mass_bins = np.logspace(-2, 3, 10)    # Log-spaced bins for mass (0.01 to 10000 Earth masses)

# Create a 2D histogram of planet counts
counts, xedges, yedges = np.histogram2d(final_positions, final_masses, bins=[radius_bins, mass_bins])

# Normalize counts to percentages
print(planet_dicts)
total_planets = planet_dicts[0]['tot_planets']
print("Total planets:", total_planets)
percentages = (counts / total_planets) * 100

# Plot the 2D histogram as a color-coded grid
fig, ax = plt.subplots(figsize=(10, 8))
mesh = ax.pcolormesh(xedges, yedges, counts.T, cmap='viridis', norm=LogNorm(vmin=np.min(counts[counts>0]), vmax=np.max(counts)))  # Transpose counts to match X, Y
cbar = plt.colorbar(mesh, ax=ax)
cbar.set_label('Number of Planets', fontsize=20)  # Increase colorbar label size
cbar.ax.tick_params(labelsize=20) 
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)
# Add percentage text to each bin
for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        if counts[i, j] > 0:  # Only display text for non-empty bins
            ax.text(
                (xedges[i] + xedges[i + 1]) / 2,  # x-coordinate (bin center)
                (yedges[j] + yedges[j + 1]) / 2,  # y-coordinate (bin center)
                f"{percentages[i, j]:.1f}%",      # Percentage text
                color="white",
                ha="center",
                va="center",
                fontsize=8
            )

#plot.boxes(ax)
# Set axis scales and labels
ax.set_ylim(1e-2, 1e3)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('r [AU]', fontsize=25, labelpad=20)
ax.set_ylabel('M [$M_{\oplus}$]', fontsize=25, labelpad=20)
ax.set_title('Planet counts', fontsize=20)
ax.tick_params(axis="both", which="major", direction='out', size=15, labelsize=18)
ax.tick_params(axis="both", which="minor", direction='out', size=10)
plt.tight_layout()
plt.savefig("figures/pop_synth/planet_radius_mass_distribution_surfheat_randomMstar_randomZ", bbox_inches='tight')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# final positions and masses of the simulation loaded above
final_positions = []
final_masses = []
R_E = []
mass_ratio = []
star_masses = []
D_s = 8
D_L = 4
for j in range(len(simulations[0])):
    sim = simulations[0][j]
    params = parameters[0][j]
    sim_params = sim_parameters[0][j]
    for p in range(sim_params.nr_planets):
        idx = plot.idxs (axs, sim.time, sim.mass[p], sim.position[p], sim.filter_fraction[p], 
                        sim.dR_dt[p], sim.dM_dt[p], params, True)
        iso_idx = np.argmax(sim.mass[p] > M_peb_iso(sim.position[p].value, sim.time.value, params)*u.M_earth)
        stop_mig_idx = idx['stop_mig_idx'].values[0]

        final_positions.append(sim.position[p,stop_mig_idx].to(u.au).value)
        final_masses.append(sim.mass[p,stop_mig_idx].to(u.M_earth).value) 
        mass_ratio.append(sim.mass[p,stop_mig_idx].to(u.M_earth).value/(params.star_mass))
        star_masses.append(params.star_mass)
        R_E.append(R_Einstein(D_s, D_L, params))


# Define the grid bins for radius and mass
radius_bins = np.logspace(-2, 2, 10)  # Log-spaced bins for radius (0.01 to 100  AU)
mass_bins = np.logspace(-7, -5, 10)    # Log-spaced bins for mass (0.01 to 10000 Earth masses)

# Create a 2D histogram of planet counts
counts, xedges, yedges = np.histogram2d(np.array(final_positions)/np.array(R_E),np.array(mass_ratio), bins=[radius_bins, mass_bins])

# Normalize counts to percentages
print(planet_dicts)
total_planets = planet_dicts[0]['tot_planets']
print("Total planets:", total_planets)
percentages = (counts / total_planets) * 100

# Plot the 2D histogram as a color-coded grid
fig, ax = plt.subplots(figsize=(10, 8))
mesh = ax.pcolormesh(xedges, yedges, counts.T, cmap='viridis', norm=LogNorm(vmin=np.min(counts[counts>0]), vmax=np.max(counts)))  # Transpose counts to match X, Y
cbar = plt.colorbar(mesh, ax=ax)
cbar.set_label('Number of Planets', fontsize=20)  # Increase colorbar label size
cbar.ax.tick_params(labelsize=20) 
cbar.ax.tick_params(axis="both", which="major", size=13, labelsize=13)
cbar.ax.tick_params(axis="both", which="minor", size=9)
# Add percentage text to each bin
for i in range(len(xedges) - 1):
    for j in range(len(yedges) - 1):
        if counts[i, j] > 0:  # Only display text for non-empty bins
            ax.text(
                (xedges[i] + xedges[i + 1]) / 2,  # x-coordinate (bin center)
                (yedges[j] + yedges[j + 1]) / 2,  # y-coordinate (bin center)
                f"{percentages[i, j]:.1f}%",      # Percentage text
                color="white",
                ha="center",
                va="center",
                fontsize=8
            )
q = np.geomspace(10**(-4.5), 1e-2)
ax.loglog(lensing_triangle(q)[0], q, color = 'black')
ax.loglog(lensing_triangle(q)[1], q, color = 'black')
#plot.boxes(ax)
# Set axis scales and labels
ax.set_ylim(8e-8, 1e-2)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('s [$R_{\mathrm{Einstein}}$]', fontsize=25, labelpad=20)
ax.set_ylabel('q', fontsize=25, labelpad=20)
ax.set_title('Planet counts', fontsize=20)
ax.tick_params(axis="both", which="major", direction='out', size=15, labelsize=18)
ax.tick_params(axis="both", which="minor", direction='out', size=10)
plt.tight_layout()
plt.savefig("figures/pop_synth/planet_radius_mass_distribution_surfheat_randomMstar_randomZ", bbox_inches='tight')

fig, axs = plt.subplots(1, 1, figsize=(8,8))
Mstars = np.array(star_masses)*u.M_earth.to(u.M_sun)
print("mstar",Mstars)
bins = np.logspace(np.log10(min(Mstars)), np.log10(max(Mstars)), 20)
plt.hist(Mstars, bins = bins, edgecolor='black', color = 'red', density = True)
plt.xscale("log")  # because x is logspaced
plt.title("Distribution of Stellar Masses")
plt.xlabel("Stellar Mass ($M_\odot$)")
plt.ylabel("Probability Density")

# Calculate the mean
mean = np.mean(Mstars)
median = np.median(Mstars)
from scipy.stats import gaussian_kde

# Calculate the standard deviation
std_dev = np.std(Mstars) 
# Perform kernel density estimation
kde = gaussian_kde(Mstars)
x = np.linspace(min(Mstars), max(Mstars), 1000)  # Generate x values for the KDE
density = kde(x)

# Find the peak of the KDE
peak_index = np.argmax(density)
peak_value = x[peak_index]
# Print the results
print(f"Peak Value (Mode): {peak_value}")
# Print the results
print(f"Mean: {mean}")
print(f"Median: {median}")
print(f"Standard Deviation: {std_dev}")
plt.savefig('figures/pop_synth/mass_disc_distribution', bbox_inches='tight')
